# **This notebook is only used to submit LB, riiid_dataset contains the trained model and final user state, this notebook does not include the training process!**


* **This is my first time participating in a Kaggle Competition（Formal）. I am currently a freshman studying a non-computer major  (just 3 months after entering the colledge) and I have never participated in any competition in high school before. Most of my Python skills are learnt online. Therefore, the code style/model description may be very confusing, Sorry for the inconvenience. If you have any suggestions/questions, you are welcome to point out in the comment section below**

* This notebook is for submission ONLY. The training code is quiet messy and needs to be sorted out. I may upload it if I have time in the near future.

* This notebook is mostly translated by Google Translation,as English is not my native language.



* **Notes on riiid-dataset dataset**


1. Model（classifier.model）

Classifier.model is a model trained by LightGBM without parameter optimization. All parameters are LightGBM default (random seed is 2020), and it is only trained on the first 20M rows of data.If parameters are optimized/trained on the entire data set, this model may be still  room for improvement.

2. Stage.Pickle

Considering  the feature engineering is obtained by iterating the DataFrame, and this process takes a long time, in order to speed up the notebook ,we save the final state after the iteration and store it in the Pickle file.

3. Lesson_Metadata and Question_Metadata

In the Lesson_Metadata and Question_Metadata files, we save the pre-calculated features related to the test questions (Tag, Mean Correct rate, etc.) and the features related to the course (Tag).


* **Feature Description**


1. question_answered_num 

The total number of questions answered by users


2. question_answered_num_agg_field 

The total question answered by the user (according to TOEIC subject area statistics)


3. question_answered_mean_accuracy 

The average correct rate of questions answered by users


4. question_answered_mean_accuracy_agg_field 

The average correct rate of questions answered by users (according to TOEIC subject area statistics)


5.question_answered_mean_difficulty_weighted_accuracy  

The average difficulty weighted correct rate of questions answered by users


6. question_answered_mean_difficulty_weighted_accuracy_agg_field 

The average difficulty weighted correct rate of the questions answered by users (according to TOEIC subject area statistics)


7. max_solved_difficulty 

The most difficult question answered by users



8. max_solved_difficulty_agg_field 

The most difficult question answered by users (according to TOEIC subject area statistics)



9. min_wrong_difficulty 

The simplest problem that users do wrong



10. min_wrong_difficulty_agg_field 

The simplest problem that users do wrong (according to TOEIC subject area statistics)



11. lessons_overall

How many lessons the user learned in total



12. lessons_overall_agg_field 

How many lessons the user has learned in total (by TOEIC subject area statistics)



13. session_time 

Time since starting of user's session (in minutes)



14. since_last_session_time 

Time since the last session (in hours)



15. user_id 

user ID



16. tag_1 

Question Tag1 



17. tag_2 

Question Tag2 



18. tag_encoded 

encoded number based on Tag1, 2, 3



19. question_id 

The content_id of the question



20. previous_explained 

Has the previous question been answered?



21. question_seen 

Has the user ever seen this question?



22. mean_question_accuracy 

The median accuracy of the question



23. std_question_accuracy 

The standard deviation of the correct rate of the question


24. most_liked_guess_correct 

Suppose you can't do a certain multiple-choice question in the exam, then you will probably guess a C (I'm used to guessing the longer choice) or something. In view of this, we record the user’s favorite option every time a user makes a mistake. After that, if the Trueskill model mentioned below believes that the probability of the user doing the question correctly is low, we will compare the user's favorite option with the option of the question to see if the user guessed it correctly.



*The remaining features are related to the Trueskill scoring system*

Trueskill is a scoring system developed by Microsoft (https://www.microsoft.com/en-us/research/project/trueskill-ranking-system/)

If you have played Rainbow Six, you may have been exposed to this system. In this game, your ranking is determined by the Trueskill system.

Consider regarding each Question as a user (experienced player) with determined abilities, and User as a novice who just started playing. After that, the Trueskill system can be used to determine its ranking and its winning probability.

*Note The reason why we can apply the Trueskill system is because the distribution of the difficulty of the problem is close to a normal distribution [determined by (1-the median accuracy of the user group as a whole)].

25. mmr_overall

user's MMR score

26. mmr_overall_agg_field 

user's MMR score (according to TOEIC subject area statistics)

27. mmr_confidence 

User's MMR confidence level

28. mmr_overall_agg_field 

user's MMR confidence level (according to TOEIC subject area statistics)

29. mmr_win_prob 

The probability of the user'beating' the question

30. mmr_win_prob_agg_field 

The probability of the user'beating' the question (based on TOEIC subject area statistics)




* **Copyright Statements**

Trueskill(TM) is a registered trademark of Microsoft Corporation.

The truskill library was created by Heungsub Lee, based on the BSD open source project, check out: https://trueskill.org/

In [ ]:
'''
Riiid Competition Submission Ver 1.0.1 Alpha
(C) Copyright By Author 2020 - Now
All rights reserved
'''
import sys
sys.path.append('/kaggle/input/riiid-dataset/')
#Dir
question_metadata_dir = r'/kaggle/input/riiid-dataset/question_metadata.csv'
lesson_metadata_dir = r'/kaggle/input/riiid-dataset/lesson_metadata.csv'
pickle_dir= r'/kaggle/input/riiid-dataset/stage.pickle'
model_dir = r'/kaggle/input/riiid-dataset/classifier.model'

import datetime
print("{} Submission Started".format(str(datetime.datetime.now())))
# Import some Stuff
try:
    import pandas as pd
    import pickle
    import trueskill
    import math
    import lightgbm as lgb
    import riiideducation
    import time
    from sklearn.metrics import roc_auc_score


except ImportError as e:
    print("{} Import Processed finished with error：{}".format(str(datetime.datetime.now()), e))

print("{} Import Completed".format(str(datetime.datetime.now())))
env = trueskill.TrueSkill(mu=0.3, sigma=0.164486, beta=0.05, tau=0.00164, draw_probability=0)
env.make_as_global()


def win_probability(team1, team2):
    '''
    Calculate the win possibility based on two Trueskill objects
    :param team1:User TrueSkill Object
    :param team2:Question Trueskill Object
    :return: Winning Prob
    '''
    delta_mu = team1.mu - team2.mu
    sum_sigma = sum([team1.sigma ** 2, team2.sigma ** 2])
    size = 2
    denom = math.sqrt(size * (0.05 * 0.05) + sum_sigma)
    ts = trueskill.global_env()
    return ts.cdf(delta_mu / denom)


class user:
    '''
    User Class
    '''

    def __init__(self):
        '''
        Init User Class
        :param None
        :return: None
        '''
        # Please Refer to Documentation above for meaning of features.
        # Directly output features

        # Counting 
        self.question_answered_num = 0  
        self.question_answered_num_agg_field = [0] * 7 

        # Correct Rate
        self.question_answered_mean_accuracy = 0  
        self.question_answered_mean_accuracy_agg_field = [0] * 7  
        self.question_answered_mean_difficulty_weighted_accuracy = 0  
        self.question_answered_mean_difficulty_weighted_accuracy_agg_field = [0] * 7  
        # Min/Max stuff
        self.max_solved_difficulty = 1
        self.max_solved_difficulty_agg_field = [1] * 7
        self.min_wrong_difficulty = 0 
        self.min_wrong_difficulty_agg_field = [0] * 7  

        # Lessons stuff
        self.lessons_overall = 0 
        self.lessons_overall_agg_field = [0] * 7  

        # Session timing 
        self.session_time = 0  
        self.since_last_session_time = 0  

        # Features need some processing
        self._mmr_object = trueskill.setup(mu=0.3, sigma=0.164486, beta=0.05, tau=0.00164,
                                           draw_probability=0).Rating() 
        self._mmr_object_agg_field = [trueskill.setup(mu=0.3, sigma=0.164486, beta=0.05, tau=0.00164,
                                                      draw_probability=0).Rating()] * 7
        self._most_liked_guess = [0] * 4 
        self._last_session_start_time = 0  
        self._first_action_time = 0  
        self._question_num_dict = {}  
        self._first_processed_flag = False  

    def update_user(self, data: pd.DataFrame):
        '''
        Update user with one row of DataFrame
        :param data: pandas DataFrame
        :return: None
        '''
        _temp = None

        # Judging whether user are watching courses
        if data['content_type_id'] == 0:
            # Content Type = 0,means User are answering Questions.

            # Counting Part
            self.question_answered_num = self.question_answered_num + 1
            question_field = int(data['content_field'])
            self.question_answered_num_agg_field[question_field - 1] = int(self.question_answered_num_agg_field[
                                                                               question_field - 1]) + 1

            # Average Correct Rate
            if data['answered_correctly'] == 1:
                self.question_answered_mean_accuracy = \
                    (self.question_answered_mean_accuracy * (
                            self.question_answered_num - 1) + 1) / self.question_answered_num

                self.question_answered_mean_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1) + 1) \
                    / self.question_answered_num_agg_field[question_field - 1]

                self.question_answered_mean_difficulty_weighted_accuracy = \
                    (self.question_answered_mean_difficulty_weighted_accuracy * (self.question_answered_num - 1) + (
                            1 - data['mean_question_accuracy']) * 3) \
                    / self.question_answered_num

                self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1) + (
                             1 - data['mean_question_accuracy']) * 3) \
                    / self.question_answered_num_agg_field[question_field - 1]


            else:
                self.question_answered_mean_accuracy = \
                    (self.question_answered_mean_accuracy * (
                            self.question_answered_num - 1)) / self.question_answered_num

                self.question_answered_mean_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1)) / \
                    self.question_answered_num_agg_field[question_field - 1]

                self.question_answered_mean_difficulty_weighted_accuracy = \
                    (self.question_answered_mean_difficulty_weighted_accuracy * (self.question_answered_num - 1)) \
                    / self.question_answered_num

                self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] = \
                    (self.question_answered_mean_difficulty_weighted_accuracy_agg_field[question_field - 1] * (
                            self.question_answered_num_agg_field[question_field - 1] - 1)) \
                    / self.question_answered_num_agg_field[question_field - 1]

            # Min/Max Part

            if data['answered_correctly'] == 1:
                if data['mean_question_accuracy'] < self.max_solved_difficulty:
                    self.max_solved_difficulty = data['mean_question_accuracy']
                if data['mean_question_accuracy'] < self.max_solved_difficulty_agg_field[question_field - 1]:
                    self.max_solved_difficulty_agg_field[question_field - 1] = data['mean_question_accuracy']
            else:
                if data['mean_question_accuracy'] > self.min_wrong_difficulty:
                    self.min_wrong_difficulty = data['mean_question_accuracy']
                if data['mean_question_accuracy'] > self.min_wrong_difficulty_agg_field[question_field - 1]:
                    self.min_wrong_difficulty_agg_field[question_field - 1] = data['mean_question_accuracy']

            # Guessing Part
            if data['answered_correctly'] == 0:
                self._most_liked_guess[int(data['user_answer'])] = self._most_liked_guess[
                                                                       int(data['user_answer'])] + 1

            # Session Timing part
            if self._first_action_time == 0:
                self._first_action_time = data['timestamp']
                self._last_session_start_time = data['timestamp']
            else:
                if data['timestamp'] - self._last_session_start_time >= 7200 * 1000:
                    self.since_last_session_time = (data[
                                                        'timestamp'] - self._last_session_start_time) / 1000 / 3600
                    self._last_session_start_time = data['timestamp']
                    self.session_time = 0
                else:
                    self.session_time = (data['timestamp'] - self._last_session_start_time) / 1000 / 60

            # Answer history part
            if str(data['content_id']) in self._question_num_dict:
                self._question_num_dict[str(data['content_id'])] = self._question_num_dict[str(data['content_id'])] + 1
            else:
                self._question_num_dict[str(data['content_id'])] = 1

            # Trueskill part
            if data['answered_correctly'] == 1:
                self._mmr_object, _temp = \
                    trueskill.rate_1vs1(self._mmr_object,
                                        trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05, tau=0.00164, draw_probability=0).Rating())
                self._mmr_object_agg_field[question_field - 1], _temp = \
                    trueskill.rate_1vs1(self._mmr_object_agg_field[question_field - 1],
                                        trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05,
                                                        tau=0.00164, draw_probability=0).Rating())
            else:
                _temp, self._mmr_object = \
                    trueskill.rate_1vs1(trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05, tau=0.00164, draw_probability=0).Rating(),
                                        self._mmr_object)

                _temp, self._mmr_object_agg_field[question_field - 1] = \
                    trueskill.rate_1vs1(trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486,
                                                        beta=0.05,
                                                        tau=0.00164, draw_probability=0).Rating(),
                                        self._mmr_object_agg_field[question_field - 1])



        else:
            # Content Type !=0,User are watching a  lecture

            self.lessons_overall = self.lessons_overall + 1
            lesson_field = int(data['content_field'])
            self.lessons_overall_agg_field[lesson_field - 1] = self.lessons_overall_agg_field[lesson_field - 1] + 1

    def process_output(self, data):
        '''
        
         Output data according to user's existing attributes
        :param data: One row of dataset
        :return: output_dict dict data for training/predicting
        '''
        output_dict = {}

        # Counting Part
        output_dict['question_answered_num'] = self.question_answered_num
        output_dict['question_answered_num_agg_field'] = self.question_answered_num_agg_field[
            int(data['content_field']) - 1]

        # Average Correct Rate
        output_dict['question_answered_mean_accuracy'] = self.question_answered_mean_accuracy

        output_dict['question_answered_mean_accuracy_agg_field'] = self.question_answered_mean_accuracy_agg_field[
            int(data['content_field']) - 1]
        output_dict[
            'question_answered_mean_difficulty_weighted_accuracy'] = self.question_answered_mean_difficulty_weighted_accuracy
        output_dict['question_answered_mean_difficulty_weighted_accuracy_agg_field'] = \
            self.question_answered_mean_difficulty_weighted_accuracy_agg_field[int(data['content_field']) - 1]

        #  Min/Max Part

        output_dict['max_solved_difficulty'] = self.max_solved_difficulty
        output_dict['max_solved_difficulty_agg_field'] = self.max_solved_difficulty_agg_field[
            int(data['content_field']) - 1]
        output_dict['min_wrong_difficulty'] = self.min_wrong_difficulty
        output_dict['min_wrong_difficulty_agg_field'] = self.min_wrong_difficulty_agg_field[
            int(data['content_field']) - 1]

        # Lesson Learning part
        output_dict['lessons_overall'] = self.lessons_overall
        output_dict['lessons_overall_agg_field'] = self.lessons_overall_agg_field[int(data['content_field']) - 1]
        if output_dict['lessons_overall_agg_field'] > 0:
            output_dict['field_learnt'] = 1
        else:
            output_dict['field_learnt'] = 0
        # Session Timing part
        output_dict['session_time'] = self.session_time
        output_dict['time_to_last_session'] = self.since_last_session_time

        output_dict['task_id'] = data['task_container_id']
        output_dict['prior_time'] = data['prior_question_elapsed_time']
        # Question Statics part
        output_dict['mean_question_accuracy'] = data['mean_question_accuracy']
        output_dict['std_question_accuracy'] = data['std_accuracy']
        output_dict['question_id'] = data['content_id']
        # TrueSkill part
        output_dict['mmr_overall'] = self._mmr_object.mu
        output_dict['mmr_overall_agg_field'] = self._mmr_object_agg_field[int(data['content_field']) - 1].mu
        output_dict['mmr_confidence'] = self._mmr_object.sigma

        output_dict['mmr_overall_agg_field'] = self._mmr_object_agg_field[int(data['content_field']) - 1].sigma
        output_dict['mmr_win_prob'] = win_probability(self._mmr_object,
                                                      trueskill.setup(mu=1 - data['mean_question_accuracy'],
                                                                      sigma=0.164486,
                                                                      beta=0.05, tau=0.00164,
                                                                      draw_probability=0).Rating())
        output_dict['mmr_win_prob_agg_field'] = win_probability(
            self._mmr_object_agg_field[int(data['content_field']) - 1],
            trueskill.setup(mu=1 - data['mean_question_accuracy'], sigma=0.164486, beta=0.05,
                            tau=0.00164, draw_probability=0).Rating())
        output_dict['user_id'] = data['user_id']
        output_dict['tag_1'] = data['tag_1']
        output_dict['tag_2'] = data['tag_2']

        output_dict['tags_encoded'] = data['tags_encoded']
        # Other features

        if not pd.isna(['prior_question_had_explanation']):
            output_dict['previous_explained'] = data['prior_question_had_explanation']
        else:
            output_dict['previous_explained'] = False

        if str(data['content_id']) in self._question_num_dict:
            output_dict['question_seen'] = 1
        else:
            output_dict['question_seen'] = 0

        # Guessing part
        max_choice = 0
        max_choice_num = 0
        i = 0
        for item in self._most_liked_guess:
            if item > max_choice_num:
                max_choice_num = item
                max_choice = i
            i = i + 1

        if output_dict['mmr_win_prob'] <= 0.4:
            if max_choice == data['correct_answer']:
                output_dict['most_liked_guess_correct'] = True
            else:
                output_dict['most_liked_guess_correct'] = False
        else:
            output_dict['most_liked_guess_correct'] = True

        # Target
        #output_dict['answered_correctly'] = data['answered_correctly']

        return output_dict

# Import Metadata
question_metadata = pd.read_csv(question_metadata_dir)
lesson_metadata = pd.read_csv(lesson_metadata_dir)
print("{} Metadata Imported".format(str(datetime.datetime.now())))

#Indexing Metadata
question_metadata = question_metadata.set_index(keys=['content_id'])
lesson_metadata = lesson_metadata.set_index(keys=['content_id'])
print("{} Metadata Indexed".format(str(datetime.datetime.now())))

#Import pickle Object
with open(pickle_dir, 'rb') as fo:
    user_pickle = pickle.load(fo)

print("{} Pickle Object Imported".format(str(datetime.datetime.now())))

#Rebuilding Trueskill 
for user_id,user_info in user_pickle.items():
    user_pickle[user_id]._mmr_object = trueskill.setup(mu=user_pickle[user_id]._mmr_object[0],
                                                       sigma=user_pickle[user_id]._mmr_object[1],
                                                       beta=0.05, tau=0.00164,
                                                       draw_probability=0).Rating()
    for i in range(0, 7):
        # 1+1
        user_pickle[user_id]._mmr_object_agg_field[i] =  trueskill.setup(mu=user_pickle[user_id]._mmr_object_agg_field[i][0],
                                                       sigma=user_pickle[user_id]._mmr_object_agg_field[i][1],
                                                       beta=0.05, tau=0.00164,
                                                       draw_probability=0).Rating()

print("{} Pickle Trueskill Rebuilt".format(str(datetime.datetime.now())))

#Import Model
model = lgb.Booster(model_file=model_dir)
print("{} Model Imported".format(str(datetime.datetime.now())))

#Init Environment
env = riiideducation.make_env()
iter_test = env.iter_test()
print("{} Init Competiton Environment".format(str(datetime.datetime.now())))

#Init Others
rows_accum = 0 #Row Counter
first_submission = True 
model_prd = [0]
true_value = []
last_df = pd.DataFrame()
print("{} Init Done!".format(str(datetime.datetime.now())))

for (test_df, sample_prediction_df) in iter_test:
    if first_submission == False:
        last_df['answered_correctly'] = eval(test_df.iloc[0]['prior_group_answers_correct'])
        last_df['user_answer'] = eval(test_df.iloc[0]['prior_group_responses'])
        true_value.extend(eval(test_df.iloc[0]['prior_group_answers_correct']))
        for index,row in last_df.iterrows():
            user_pickle[row['user_id']].update_user(row)
    rows_accum = rows_accum + test_df.shape[0]
    if first_submission == False:
        1+1
        #print("{} Processing Row {}  ,AUC is {}".format(str(datetime.datetime.now()),rows_accum,roc_auc_score(true_value,model_prd)))
    test_df['answered_correctly'] = 0.6524
    st = float(time.time())
    # Merging and Concating
    try:
        sub_1 = test_df[test_df['content_type_id'] == False]
        sub_2 = test_df[test_df['content_type_id'] == True]
        del test_df
        sub_1 = sub_1.merge(question_metadata, on="content_id", how="left")
        sub_2 = sub_2.merge(lesson_metadata, on="content_id", how="left")
        test_df = pd.DataFrame()
        test_df = pd.concat([sub_1,sub_2])
    except Exception:
        pass

    for index, row in test_df.iterrows():
        try:
            if row['user_id'] not in user_pickle:
                user_pickle[row['user_id']] = user()
            if row['content_type_id'] == 0:
                predict_dict = user_pickle[row['user_id']].process_output(row)
                l = []
                for i,v in predict_dict.items():
                    l.append(v)
                prd_value = float(model.predict([l])[0])
                test_df.loc[test_df.row_id == row.row_id, 'answered_correctly'] = prd_value
                model_prd.append(prd_value)


        except Exception as e:
            print(e)
            pass

    time_taken = float(time.time()) - st
    print("{} Based on current speed，it would take {} minutes to complete".format(
        str(datetime.datetime.now()),int(time_taken / test_df.shape[0] * 2500000 / 60)))
    if first_submission == True:
        first_submission = False
    last_df = test_df
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
